In [1]:
import gradio as gr
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM
from neuralforecast.losses.pytorch import MAE
import plotly.io as pio
pio.renderers.default = "notebook"


/opt/conda/envs/ai_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 📌 Load Data
df = pd.read_csv("../data/Top_12_German_Companies_Financial_Data.csv")
df = df[df["Company"] == "Merck KGaA"].copy()

# Convert date column and sort for proper time-series analysis
df["Period"] = pd.to_datetime(df["Period"], format="%m/%d/%Y")
df = df.sort_values(by="Period")

# Convert revenue to numeric and rename for NeuralForecast
df["Revenue"] = pd.to_numeric(df["Revenue"], errors="coerce")
df = df.rename(columns={"Period": "ds", "Revenue": "y"})
df["unique_id"] = "all"
df = df[["unique_id", "ds", "y"]]

In [3]:
# 📌 Train-Test Split
val_size = int(len(df) * 0.2)
train, val = df[:-val_size], df[-val_size:]

In [4]:
# 📌 Define & Train the Model **(Once)**
hidden_neurons = 12
input_size = 8
best_model = LSTM(
    h=hidden_neurons, input_size=input_size, loss=MAE(), alias=f"LSTM_{hidden_neurons}"
)

# Train the model **before launching Gradio**
nf_best = NeuralForecast(models=[best_model], freq="Q")
nf_best.fit(df=train)  # ✅ Model is trained once here

Seed set to 1


Epoch 999: 100%|██████████| 1/1 [00:00<00:00, 41.95it/s, v_num=797, train_loss_step=0.259, train_loss_epoch=0.259]  


In [5]:
def forecast_turnover(horizon):
    print(f"🔍 Generating forecast for {horizon} quarters ahead...")

    horizon = int(horizon)

    # Step 1: Generate Forecast
    forecast_df = nf_best.predict().reset_index()
    print(f"🔍 Forecast DataFrame: {forecast_df.head()}")

    if forecast_df.empty:
        return None, "🚨 Error: No forecast data available!"

    # Step 2: Check Model Output Columns
    model_name = best_model.alias
    available_columns = list(forecast_df.columns)
    pred_col = next((col for col in available_columns if model_name in col), None)

    if pred_col is None:
        return None, f"🚨 Error: No valid prediction column found! Available: {forecast_df.columns}"

    print(f"✅ Using Prediction Column: {pred_col}")

    # Step 3: Check Predictions
    predictions = forecast_df[pred_col].values

    if len(predictions) < horizon:
        return None, f"🚨 Error: Model returned only {len(predictions)} predictions, expected {horizon}."

    predictions = predictions[:horizon]
    print(f"✅ Predictions: {predictions}")

    # Step 4: Ensure Future Dates Are Valid
    if train["ds"].empty:
        return None, "🚨 Error: Training data is empty!"

    last_date = train["ds"].max()
    future_dates = pd.date_range(start=last_date, periods=horizon + 1, freq="Q")[1:]
    print(f"✅ Future Dates: {future_dates}")

    # 📊 Create Plotly Figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train["ds"], y=train["y"], mode="lines", name="Training Data"))
    fig.add_trace(go.Scatter(x=val["ds"], y=val["y"], mode="lines", name="Actual Revenue"))
    fig.add_trace(go.Scatter(x=future_dates, y=predictions, mode="lines+markers", name="Forecast"))

    fig.update_layout(title="Turnover Forecast", xaxis_title="Date", yaxis_title="Revenue (€)", template="plotly_white")

    return fig, f"✅ Forecast Generated for {horizon} Quarters."


In [6]:
forecast_turnover(horizon=6)

🔍 Generating forecast for 6 quarters ahead...
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 11.51it/s]
🔍 Forecast DataFrame:   unique_id         ds       LSTM_12
0       all 2023-09-30  9.739462e+09
1       all 2023-12-31  1.746220e+10
2       all 2024-03-31  9.072552e+09
3       all 2024-06-30  1.578190e+10
4       all 2024-09-30  1.658214e+10
✅ Using Prediction Column: LSTM_12
✅ Predictions: [9.73946163e+09 1.74621962e+10 9.07255194e+09 1.57819023e+10
 1.65821420e+10 1.67350323e+10]
✅ Future Dates: DatetimeIndex(['2023-09-30', '2023-12-31', '2024-03-31', '2024-06-30',
               '2024-09-30', '2024-12-31'],
              dtype='datetime64[ns]', freq='Q-DEC')


(Figure({
     'data': [{'mode': 'lines',
               'name': 'Training Data',
               'type': 'scatter',
               'x': array(['2017-03-31T00:00:00.000000000', '2017-06-30T00:00:00.000000000',
                           '2017-09-30T00:00:00.000000000', '2017-12-31T00:00:00.000000000',
                           '2018-03-31T00:00:00.000000000', '2018-06-30T00:00:00.000000000',
                           '2018-09-30T00:00:00.000000000', '2018-12-31T00:00:00.000000000',
                           '2019-03-31T00:00:00.000000000', '2019-06-30T00:00:00.000000000',
                           '2019-09-30T00:00:00.000000000', '2019-12-31T00:00:00.000000000',
                           '2020-03-31T00:00:00.000000000', '2020-06-30T00:00:00.000000000',
                           '2020-09-30T00:00:00.000000000', '2020-12-31T00:00:00.000000000',
                           '2021-03-31T00:00:00.000000000', '2021-06-30T00:00:00.000000000',
                           '2021-09-30T00:00:00

In [7]:
iface = gr.Interface(
    fn=forecast_turnover,
    inputs=gr.Slider(minimum=1, maximum=6, step=1, label="Forecast Horizon (Quarters)"),
    outputs=[gr.Plot(), gr.Textbox()],
    title="Merck KGaA Turnover Forecast",
    description="Select the forecast horizon (in quarters) to generate turnover predictions for Merck KGaA.",
)

# ✅ Use `share=True` to avoid localhost issues
iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://3bc885da9987f8e4d9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔍 Generating forecast for 4 quarters ahead...
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 75.45it/s]
🔍 Forecast DataFrame:   unique_id         ds       LSTM_12
0       all 2023-09-30  9.739462e+09
1       all 2023-12-31  1.746220e+10
2       all 2024-03-31  9.072552e+09
3       all 2024-06-30  1.578190e+10
4       all 2024-09-30  1.658214e+10
✅ Using Prediction Column: LSTM_12
✅ Predictions: [9.73946163e+09 1.74621962e+10 9.07255194e+09 1.57819023e+10]
✅ Future Dates: DatetimeIndex(['2023-09-30', '2023-12-31', '2024-03-31', '2024-06-30'], dtype='datetime64[ns]', freq='Q-DEC')
🔍 Generating forecast for 6 quarters ahead...
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 127.38it/s]
🔍 Forecast DataFrame:   unique_id         ds       LSTM_12
0       all 2023-09-30  9.739462e+09
1       all 2023-12-31  1.746220e+10
2       all 2024-03-31  9.072552e+09
3       all 2024-06-30  1.578190e+10
4       all 2024-09-30  1.658214e+10
✅ Using Prediction Column: LSTM_12
✅ Pr